In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/1_Compartment_Analysis/Expt1.3_Cluster Distribution Analyses/Input_S4')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/1_Compartment_Analysis/Expt1.3_Cluster Distribution Analyses/Output_S4')
cur_date = "021821"

library(readxl)
library(data.table)
library(ggplot2)
library(factoextra)
library(janitor)
library(dplyr)
library(tidyverse)
library(gridExtra)
library(cluster)
library(vegan)
library(fpc)
library(ggdendro)

#reading in file
cytokines <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))
subjects = data.frame(read_excel("SubjectInfo_102920.xlsx", sheet = 2))

#cluster assignments for each compartment 
ELF_clus <- data.frame(read_excel("021421ELF_cluster_assignments.xlsx")) 
NLF_clus <- data.frame(read_excel("021421NLF_cluster_assignments.xlsx"))
Serum_clus <- data.frame(read_excel("021421Serum_cluster_assignments.xlsx"))
Sputum_clus <- data.frame(read_excel("021421Sputum_cluster_assignments.xlsx"))

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.6     ✔ purrr   0.3.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr:

Assigning the same baseline clusters to e-cig and cigarette smokers and running wilcoxon rank sum tests to see if they differ. 

# Eigencytokines

In [2]:
get_eigencytokines = function(smoking_status){
    # Separating the cytokine data into compartment dfs
    cytokines <- cytokines %>% 
        filter(Group == smoking_status) %>% #only want non-smokers for baseline analysis
        group_by(Compartment) %>% 
        group_split
    ELF <- cytokines[[1]]
    NLF <- cytokines[[2]]
    Serum <- cytokines[[3]]
    Sputum <- cytokines[[4]]

    # reshaping data 
    ELF <- reshape2::dcast(ELF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    NLF <- reshape2::dcast(NLF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Serum <- reshape2::dcast(Serum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Sputum <- reshape2::dcast(Sputum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 

    #background filter eliminating any cytokines that are not expressed in a compartment  
    NLF$I309 <- NULL
    Sputum$I309 <- NULL 

    # the scale function operates across columns 
    ELF_scaled <- ELF %>% 
      scale() %>% 
      as.data.frame()
    NLF_scaled <- NLF %>% 
      scale() %>% 
      as.data.frame()
    Serum_scaled <- Serum %>% 
      scale() %>% 
      as.data.frame()
    Sputum_scaled <- Sputum %>% 
      scale() %>% 
      as.data.frame()

    #transpose cytokine data for each compartment
    ELF <- as.data.frame(t(ELF))
    NLF <- as.data.frame(t(NLF)) 
    Serum <- as.data.frame(t(Serum))
    Sputum <- as.data.frame(t(Sputum)) 

    #renaming first column, grouping and splitting by "Cluster" column
    ELF_clus <- ELF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    NLF_clus <- NLF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Serum_clus <- Serum_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Sputum_clus <- Sputum_clus %>% 
      group_by(Cluster) %>% 
      group_split

    #making dfs for each cluster for PCA analysis 
    ELF_1 <- ELF_clus[[1]]
    ELF_2 <- ELF_clus[[2]]
    ELF_3 <- ELF_clus[[3]]

    NLF_1 <- NLF_clus[[1]]
    NLF_2 <- NLF_clus[[2]]
    NLF_3 <- NLF_clus[[3]]

    Serum_1 <- Serum_clus[[1]]
    Serum_2 <- Serum_clus[[2]]
    Serum_3 <- Serum_clus[[3]]

    Sputum_1 <- Sputum_clus[[1]]
    Sputum_2 <- Sputum_clus[[2]]
    Sputum_3 <- Sputum_clus[[3]]

    #making df with subjects' cytokine concentration data for each cluster 
    ELF_1 <- ELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% ELF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    ELF_2 <- ELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% ELF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    ELF_3 <- ELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% ELF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    NLF_1 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_2 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_3 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Serum_1 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Serum_2 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Serum_3 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Sputum_1 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_2 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_3 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    #PCA on each cluster, eigenvectors are in rotation -- PROBLEM - for some reason had to convert everything to numeric  
    pca_ELF_1 <- ELF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_ELF_2 <- ELF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_ELF_3 <- ELF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>%   
      prcomp()

    pca_NLF_1 <- NLF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_NLF_2 <- NLF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_NLF_3 <- NLF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()

    pca_Serum_1 <- Serum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_Serum_2 <- Serum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_Serum_3 <- Serum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_Sputum_1 <- Sputum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_Sputum_2 <- Sputum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()
    pca_Sputum_3 <- Sputum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp()

    #eigenvector dfs of first principal component 
    eigencytokines_ELF_1 <- data.frame(pca_ELF_1$rotation[,"PC1"])
      colnames(eigencytokines_ELF_1)[1] <- "ClusterA"
    eigencytokines_ELF_2 <- data.frame(pca_ELF_2$rotation[,"PC1"])
      colnames(eigencytokines_ELF_2)[1] <- "ClusterB"
    eigencytokines_ELF_3 <- data.frame(pca_ELF_3$rotation[,"PC1"])
      colnames(eigencytokines_ELF_3)[1] <- "ClusterC"

    eigencytokines_NLF_1 <- data.frame(pca_NLF_1$rotation[,"PC1"])
      colnames(eigencytokines_NLF_1)[1] <- "ClusterA"
    eigencytokines_NLF_2 <- data.frame(pca_NLF_2$rotation[,"PC1"])
      colnames(eigencytokines_NLF_2)[1] <- "ClusterB"
    eigencytokines_NLF_3 <- data.frame(pca_NLF_3$rotation[,"PC1"])
      colnames(eigencytokines_NLF_3)[1] <- "ClusterC"

    eigencytokines_Serum_1 <- data.frame(pca_Serum_1$rotation[,"PC1"])
      colnames(eigencytokines_Serum_1)[1] <- "ClusterA"
    eigencytokines_Serum_2 <- data.frame(pca_Serum_2$rotation[,"PC1"])
      colnames(eigencytokines_Serum_2)[1] <- "ClusterB"
    eigencytokines_Serum_3 <- data.frame(pca_Serum_3$rotation[,"PC1"])
      colnames(eigencytokines_Serum_3)[1] <- "ClusterC"

    eigencytokines_Sputum_1 <- data.frame(pca_Sputum_1$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_1)[1] <- "ClusterA"
    eigencytokines_Sputum_2 <- data.frame(pca_Sputum_2$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_2)[1] <- "ClusterB"
    eigencytokines_Sputum_3 <- data.frame(pca_Sputum_3$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_3)[1] <- "ClusterC"

    #collapse all eigencytokine dfs
    eigencytokines_ELF <- cbind(eigencytokines_ELF_1, eigencytokines_ELF_2, eigencytokines_ELF_3)
    eigencytokines_NLF <- cbind(eigencytokines_NLF_1, eigencytokines_NLF_2, eigencytokines_NLF_3)
    eigencytokines_Serum <- cbind(eigencytokines_Serum_1, eigencytokines_Serum_2, eigencytokines_Serum_3)
    eigencytokines_Sputum <- cbind(eigencytokines_Sputum_1, eigencytokines_Sputum_2, eigencytokines_Sputum_3)
    
    all_eigencytokines = cbind(eigencytokines_NLF, eigencytokines_ELF, eigencytokines_Sputum, eigencytokines_Serum)
    return(all_eigencytokines)
}

#calling fn
eigencytokines_NLF_NS = get_eigencytokines("NS")[1:3]
eigencytokines_ELF_NS = get_eigencytokines("NS")[4:6]
eigencytokines_Sputum_NS = get_eigencytokines("NS")[7:9]
eigencytokines_Serum_NS = get_eigencytokines("NS")[10:12]
eigencytokines_NLF_Ecig = get_eigencytokines("Ecig")[1:3]
eigencytokines_ELF_Ecig = get_eigencytokines("Ecig")[4:6]
eigencytokines_Sputum_Ecig = get_eigencytokines("Ecig")[7:9]
eigencytokines_Serum_Ecig = get_eigencytokines("Ecig")[10:12]
eigencytokines_NLF_CS = get_eigencytokines("CS")[1:3]
eigencytokines_ELF_CS = get_eigencytokines("CS")[4:6]
eigencytokines_Sputum_CS = get_eigencytokines("CS")[7:9]
eigencytokines_Serum_CS = get_eigencytokines("CS")[10:12]

In [12]:
#exporting eigencytokines
eigencytokines_NLF = rbind(eigencytokines_NLF_NS, eigencytokines_NLF_CS, eigencytokines_NLF_Ecig)
eigencytokines_ELF = rbind(eigencytokines_ELF_NS, eigencytokines_ELF_CS, eigencytokines_ELF_Ecig)
eigencytokines_Sputum = rbind(eigencytokines_Sputum_NS, eigencytokines_Sputum_CS, eigencytokines_Sputum_Ecig)
eigencytokines_Serum = rbind(eigencytokines_Serum_NS, eigencytokines_Serum_CS, eigencytokines_Serum_Ecig)

write.csv(eigencytokines_NLF, paste0(Output,"/", cur_date, "_NLF_eigencytokines.csv"), row.names = TRUE)
write.csv(eigencytokines_ELF, paste0(Output,"/", cur_date, "_ELF_eigencytokines.csv"), row.names = TRUE)
write.csv(eigencytokines_Sputum, paste0(Output,"/", cur_date, "_Sputum_eigencytokines.csv"), row.names = TRUE)
write.csv(eigencytokines_Serum, paste0(Output,"/", cur_date, "_Serum_eigencytokines.csv"), row.names = TRUE)

# Wilcoxon Rank Sum tests

Comparing eigencytokines of non-smokers to smokers. 

In [3]:
#converting subject ids to col, melting, and adding compartment
changed_df = function(df, compartment_name){
    df = reshape2::melt(df %>%
        rownames_to_column(var = "SubjectID"), variable = "Cluster",  value.name = 'Conc_pslog2')
    df$Compartment = rep(compartment_name, times = length(df$SubjectID))
    return(df)
}
NS_eigencytokines_NLF = changed_df(eigencytokines_NLF_NS,"NLF")
NS_eigencytokines_ELF = changed_df(eigencytokines_ELF_NS,"ELF")
NS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_NS,"Sputum")
NS_eigencytokines_Serum = changed_df(eigencytokines_Serum_NS,"Serum")
Ecig_eigencytokines_NLF = changed_df(eigencytokines_NLF_Ecig,"NLF")
Ecig_eigencytokines_ELF = changed_df(eigencytokines_ELF_Ecig,"ELF")
Ecig_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_Ecig,"Sputum")
Ecig_eigencytokines_Serum = changed_df(eigencytokines_Serum_Ecig,"Serum")
CS_eigencytokines_NLF = changed_df(eigencytokines_NLF_CS,"NLF")
CS_eigencytokines_ELF = changed_df(eigencytokines_ELF_CS,"ELF")
CS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_CS,"Sputum")
CS_eigencytokines_Serum = changed_df(eigencytokines_Serum_CS,"Serum")

#recombining into 3 dataframes
NS_eigencytokines = rbind(NS_eigencytokines_NLF, NS_eigencytokines_ELF, NS_eigencytokines_Sputum, 
                          NS_eigencytokines_Serum)
CS_eigencytokines = rbind(CS_eigencytokines_NLF, CS_eigencytokines_ELF, CS_eigencytokines_Sputum, 
                          CS_eigencytokines_Serum)
Ecig_eigencytokines = rbind(Ecig_eigencytokines_NLF, Ecig_eigencytokines_ELF, Ecig_eigencytokines_Sputum, 
                          Ecig_eigencytokines_Serum)

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables



In [4]:
head(NS_eigencytokines)

,SubjectID,Cluster,Conc_pslog2,Compartment
,<chr>,<fct>,<dbl>,<chr>
1,NS_1,ClusterA,-0.30518544,NLF
2,NS_10,ClusterA,-0.21987346,NLF
3,NS_11,ClusterA,-0.09991734,NLF
4,NS_12,ClusterA,-0.30907080,NLF
5,NS_13,ClusterA,-0.39944617,NLF
6,NS_14,ClusterA,-0.46223028,NLF


In [5]:
#creating vectors to loop through
compartment = c('NLF','ELF','Sputum','Serum')
cluster = unique(NS_eigencytokines_NLF$Cluster)

#initializing vectors to store values
CS_vector = c()
Ecig_vector = c()
wilcoxon_rank_sum_values = function(df1, df2, empty_vector){
    #running wilcoxon rank sum and storing the statistic, compartment, cluster, and p value in a vector
    for (i in 1:length(compartment)){
        for (j in 1: length(cluster)){
            variable1_df = df1 %>% # baseline df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)
            variable2_df = df2 %>% # smoker df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)


            #running wilcoxon rank sum
            wilcox_test = wilcox.test(variable1_df$Conc_pslog2, variable2_df$Conc_pslog2)

            #contains compartment, cluster, u stat, p value
            values_vector = c(compartment[i], as.character(cluster[j]), wilcox_test$statistic, wilcox_test$p.value)
            empty_vector = c(empty_vector, values_vector)

        }
    }
    return(empty_vector)
}

#calling fn
CS_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, CS_eigencytokines, CS_vector)
Ecig_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, Ecig_eigencytokines, Ecig_vector)

In [6]:
final_table = function(vector){
    #"""
    #Reformatting each vector into matrices to export. 
    
    #:param: vector, demographic variable
    #:output: a 5x12 matrix containing compartment, cluster, protein, u stat, p value, p adj

    #"""
    dim(vector) = c(4, 12)
    table = data.frame(t(vector))
    colnames(table) = c("Compartment",'Cluster', 'Stat', 'P Value')
    
    #going back to calculate padj values
    PAdj = c()
    for (i in 1:length(compartment)){
        single_compartment_df = table %>%
            filter(Compartment == compartment[i])
        padj =  p.adjust(as.numeric(as.character(single_compartment_df[,4])), method = "fdr") 
        PAdj = c(PAdj, padj)
        
    }
    
    table = cbind(table, PAdj)
    return(table)
}

#calling fn
CS_table = final_table(CS_wilcoxon_values)
CS_table
Ecig_table = final_table(Ecig_wilcoxon_values)
Ecig_table

Compartment,Cluster,Stat,P Value,PAdj
<fct>,<fct>,<fct>,<fct>,<dbl>
NLF,ClusterA,102,0.615950603989371,1.000000e+00
NLF,ClusterB,91,1,1.000000e+00
NLF,ClusterC,97,0.792955235488551,1.000000e+00
ELF,ClusterA,83,0.720314184153193,1.000000e+00
ELF,ClusterB,101,0.650035546382296,1.000000e+00
ELF,ClusterC,91,1,1.000000e+00
Sputum,ClusterA,170,2.71209424527502e-05,4.068141e-05
Sputum,ClusterB,0,9.97093472527582e-08,2.991280e-07
Sputum,ClusterC,94,0.905062742106759,9.050627e-01


Compartment,Cluster,Stat,P Value,PAdj
<fct>,<fct>,<fct>,<fct>,<dbl>
NLF,ClusterA,0,7.54197509809517e-09,1.131296e-08
NLF,ClusterB,238,7.54197509809517e-09,1.131296e-08
NLF,ClusterC,176,0.0233421187915757,2.334212e-02
ELF,ClusterA,128,0.739469578547832,7.394696e-01
ELF,ClusterB,101,0.492781317320966,7.391720e-01
ELF,ClusterC,150,0.229670427943923,6.890113e-01
Sputum,ClusterA,98,0.42129653905362,5.970708e-01
Sputum,ClusterB,105,0.597070760978688,5.970708e-01
Sputum,ClusterC,98,0.42129653905362,5.970708e-01


In [7]:
#exporting tables
write.csv(CS_table, paste0(Output,"/", cur_date, "_CS_Distribution_Analysis.csv"), row.names = FALSE)
write.csv(Ecig_table, paste0(Output,"/", cur_date, "_Ecig_Distribution_Analysis.csv"), row.names = FALSE)